In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import math
import quandl
import collections, functools, operator
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from operator import itemgetter
pd.set_option('display.max_columns', 500)
pd.set_option('mode.chained_assignment', None)
%matplotlib inline

In [2]:
def data_cleaning(data):
    '''
    @author: Wuding Li
    @The function cleans the data
    '''
    
    data = data[data['date'] != 19251231]
    #counts = data['date'].value_counts().to_dict()
    
    #adjust date format
    data['date'] = data['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    
    
    #remove PRC and SHROUT missing. According to the describtion, PRC missing meanings totally missing without a clue
    data['PRC'] = pd.to_numeric(data.PRC, errors='coerce')
    data = data.dropna(subset = ['PRC'])
    data['PRC'] = data['PRC'].abs()
    
    data['SHROUT'] = pd.to_numeric(data.SHROUT, errors='coerce')
    data = data.dropna(subset = ['SHROUT'])
    
    #get the medium
    NYSE = pd.read_excel('NYSE medium.xlsx')
    NYSE['date'] = pd.to_datetime(NYSE['date'])
    data['YearMonth'] = data['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE['YearMonth'] = NYSE['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE = NYSE.drop(columns=['date'])    
    data = pd.merge(data,NYSE,how = 'left',on = 'YearMonth')
    data = data[data['YearMonth'] != '192512']
    #counts = data['date'].value_counts().to_dict()
    

    data['mkt_cap'] = data['PRC']*data['SHROUT']
    data = data[data['mkt_cap']*1000>data['market cap']] #SHROUT is approximate to 10**3

    
    #Remove ADR CLosed end fund and foreign fund
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.ffill())
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.bfill())
    data = data[data['SHRCD'].isin([11.0,10.0])]
    
    
    #check more than 5
    mask = (data['PRC'] > 5)
    data = data.loc[mask]

    #clean the RET and RETX column. 'C' stands for the first trading month. Then we change it to 0
    data['RET'] = data['RET'].apply(pd.to_numeric, errors='coerce').fillna(0)
    data['RETX'] =data['RETX'].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    data = data.drop(columns=['YearMonth'])


    return data

In [3]:
def rolling_time_window(Start_Month, Start_Year, Time_Period_Training, Time_Period_Testing, Gap, data):
    '''
    @author: Wuding Li
    @The function returns two cleaned datasets. One is the sampling period data and the other is testing period data
    @variables: Start_Month: the starting month of our rolling time period
                Start_Year: the starting year of our rolling time period
                Time_Period_Training: Span of sampling Period
                Time_Period_Testing: Span of testing Period
                Gap: Gap between sampling and testing
                Data: The input data dataset
    @Also this function apply the filter no closed end fund and stock price more than 5 dollars
    
    @Updated: 10/7 fixed the end_date_test "frequency = 0" problem
    @Updated: 10/16 change "DATE" to "date" to match the updated format
    @Updated: 11/1 used the French's medium to substitute our formal medium which directly get from dataset
    '''
    
    #get the start and end date of data
    start_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=1, freq='D')[0].date())
    end_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=2, freq=str(Time_Period_Training-1)+'M')[-1].date())
    mask_train = (data['date'] >= start_date_train) & (data['date'] <= end_date_train)
    
    
    start_month_testing = Start_Month+Time_Period_Training+Gap
    if start_month_testing % 12 == 0:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)-1
        start_month_testing = 12
    else:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)
        start_month_testing = start_month_testing % 12    
    start_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq='D')[0].date())
    end_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq=str(Time_Period_Testing)+'M')[-1].date())
    mask_test = (data['date'] >= start_date_test) & (data['date'] <= end_date_test)
    
    #applying filter functions below on the master dataset
    mask_master = (data['date'] >= start_date_train) & (data['date'] <= end_date_test)
    data_master = data.loc[mask_master]
    

    full = []
    for i in data_master['PERMNO'].values:
        if (data_master[data_master['PERMNO'] == i]).shape[0] == (Time_Period_Training+ Time_Period_Testing+ Gap):
            full.append(i)
    data_master = data_master[data_master['PERMNO'].isin(full)]

    
    return data_master.loc[mask_train],data_master.loc[mask_test]


In [4]:
def format_FF():
    '''
    @author: Zhikang Wang
    @The function read and clean up the fama_french dataset pulled from Dr. French's website
    @Variable: input is fama_french dataset pulled from Dr. French's website
    '''
    
    ff_data = pd.read_csv("fama_french.csv")
    # Getting and renaming the columns we need
    ff_ret = ff_data[['YearMonth','Mkt-RF','SMB','HML','RF']]
    ff_ret = ff_ret.rename(columns = {"Mkt-RF": "mkt_ret"})
    # converting the factor exposures to percentage
    ff_ret["mkt_ret"] = ff_ret["mkt_ret"]/100
    ff_ret["SMB"] = ff_ret["SMB"]/100
    ff_ret["HML"] = ff_ret["HML"]/100
    ff_ret["RF"] = ff_ret["RF"]/100
    ff_ret["YearMonth"] = ff_ret["YearMonth"].astype(str) #converting YearMonth to string
    
    return ff_ret

In [5]:
def get_weight(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)  
    @update: 11/17/2019 Wuding Li updated the weight function by Lehmann(1990)
    '''
    #count = len(set(data['PERMNO'].values))  
    data['weight_temp'] = data.groupby(['date'])['RET'].apply(lambda x: (x-x.mean()))
    data['weight_temp1']= data['weight_temp'].apply(lambda x: 0 if x<0 else x)
    data['weight_temp1'] = data['weight_temp1'].groupby(data['date']).transform('sum')
    data['weight'] = data['weight_temp']/data['weight_temp1']
    data = data.drop(columns=['weight_temp','weight_temp1'])
    data['weight'] = data.groupby(['PERMNO'])['weight'].shift(1)
    data = data.dropna(subset = ['weight'])
    return data

In [6]:
def get_weight_Residual(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the residual's weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)
    @update: 11/22/2019 Robin Lam and Yulin Chen updated the weight function by Lehmann(1990)
    '''
    
    count = len(set(data['PERMNO'].values))
    def resd_avg(df):
        result = (df['residual'] - df['residual_mean'])
        return result
    
    temp_output = pd.DataFrame()
    YearMonth_list = list(set(data['YearMonth'].values.tolist()))
    for i in YearMonth_list:
        temp = data[data['YearMonth'] == i]
        temp['residual_weight'] = (temp['residual'] - temp['residual_mean'])
        temp['residual_weight_winner'] = temp['residual_weight'].apply(lambda x: 0 if x<0 else x)
        temp['residual_real_weight'] = temp['residual_weight']/(temp['residual_weight_winner'].sum())
        temp = temp.drop(columns=['residual_weight','residual_weight_winner'])
        temp = temp.rename(columns = {"residual_real_weight": "residual_weight"})
        temp_output = temp_output.append(temp)
        
    temp_output = temp_output.sort_values(by=['YearMonth'])
    temp_output['residual_weight'] = temp_output.groupby(['PERMNO'])['residual_weight'].shift(1)
    temp_output = temp_output.dropna(subset = ['residual_weight'])
    return temp_output

In [7]:
def get_equal_weight(data):
    '''
    @author: Zhikang Wang
    @The function add a column to the data set that assigned equal weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    count = len(set(data['PERMNO'].values))
    data['1'] = [1]*len(data)
    data['equal_weight']=data.groupby(['date'])['1'].apply(lambda x: x*(1/count)).shift(1)
    data = data.dropna()
    return data

In [8]:
def regression_model(combined_temp):
    '''
    @author: Robin Lam, Yulin Chen, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        combined_temp: dataframe which contains returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        combined_temp: same dataframe as the input, but with an extra column (residual)
    
    @Update: 10/16/2019: Wuding Li Change the output date to match the paper 
    '''
    x = combined_temp[['mkt_ret','SMB','HML']]
    y = combined_temp['ticker_excess_ret']
    
    # Linear regression model
    x = sm.add_constant(x)
    regressor = sm.OLS(y, x).fit()
    # retrieving the slope
    coef = regressor.params
    # retrieving the t-statistics
    t_test = regressor.pvalues
    # retrieving the r-squared
    r_squared = regressor.rsquared_adj
    
    
    # calculating the residuals for each regression model
    combined_temp['residual'] = y - x['mkt_ret']*coef[1] - x['SMB']*coef[2] - x['HML']*coef[3] - coef[0]
    standard_dev = combined_temp['residual'].std() #get the sd of the residual
    avg_ret = combined_temp['residual'].mean()
    combined_temp['residual_sd'] = (combined_temp['residual']-avg_ret)/standard_dev
    combined_temp['residual_mean'] = combined_temp['residual_sd'].mean()
    
    combined_temp['mkt_factor'] = x['mkt_ret']*coef[1]    #used in conditiona_factor_model
    combined_temp['SMB_factor'] = x['SMB']*coef[2]    #used in conditiona_factor_model
    combined_temp['HML_factor'] = x['HML']*coef[3]    #used in conditiona_factor_model
    
    # adding coefficients to columns
    combined_temp['alpha'] = coef[0]
    combined_temp['mkt_beta'] = coef[1]
    combined_temp['SMB_beta'] = coef[2]
    combined_temp['HML_beta'] = coef[3]
    combined_temp['r_squared'] = r_squared
    
    # adding t-statistics to columns
    combined_temp['t_alpha'] = t_test[0]
    combined_temp['t_mkt_beta'] = t_test[1]
    combined_temp['t_SMB_beta'] = t_test[2]
    combined_temp['t_HML_beta'] = t_test[3]
    
    
    combined_temp['port_mkt_beta'] = combined_temp['mkt_beta']*combined_temp['weight']
    combined_temp['port_SMB_beta'] = combined_temp['SMB_beta']*combined_temp['weight']
    combined_temp['port_HML_beta'] = combined_temp['HML_beta']*combined_temp['weight']
    
      
#     keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
#     output = combined_temp[combined_temp['YearMonth'].isin(keep)]
    return combined_temp, standard_dev

In [9]:
def all_regression(df):
    '''
    @author: Robin Lam, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        df: dataframe which contains ALL of the stocks' returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        output: same dataframe as the input, but with an extra column (residual)
        dict_coef: dictionary of coefficients from all the regression for ALL stocks. should have 3 betas.
        
    @Update: 10/16/2019: Wuding Li Change the output format of the function and the structure of the function
    '''
    output = pd.DataFrame()
    permno_list = list(set(df['PERMNO'].values.tolist()))
    sd_dic = {}
    for i in permno_list:
        temp_train = df[df['PERMNO'] == i]
        combined_temp, sd = regression_model(temp_train)    #there are 36 months in temp_train (individual ticker)
        keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
        output_keep = combined_temp[combined_temp['YearMonth'].isin(keep)]
        sd_dic[i] = sd
        output = output.append(output_keep)
    output = get_weight_Residual(output)   #this is also returned to merge with the test dataset for later!

    return sd_dic, output

In [13]:
msf = pd.read_csv('msf.csv')
msf_before = msf

In [14]:
msf = data_cleaning(msf)

In [36]:
'''
Author: Zhikang Wang / Yulin Chen
MAIN FUNCTION.
Loop over the period from 1986 to generate rolling regression
'''

test_port = pd.DataFrame() #container for test df
total_train_output = pd.DataFrame()

dates = []
ff = []
start = 1926      #1926

for i in range(890,1086): #（7，348） （348，659） （659，890）  (890，1086）
    if (i%12) != 0:
        month = i%12
        year = i // 12 + start
    if (i%12) == 0: 
        month = 12
        year = i // 12 + start -1

    ff_ret = format_FF()
    train,test = rolling_time_window(month, year, 37, 1, 0, msf)
    #train = get_equal_weight(train)
    train = get_weight(train)
    
    train = train.rename(columns = {"RET": "ticker_ret", "date": "YearMonth"})
    train = train[['YearMonth','PERMNO','ticker_ret','weight']]
    train["PERMNO"] = train["PERMNO"].astype(str)
    test["PERMNO"] = test["PERMNO"].astype(str)
    train["YearMonth"] = train["YearMonth"].astype(str)
    train['YearMonth'] = train['YearMonth'].apply(lambda x: x[:4] + x[5:7])
    # Merging the two dataset into one combined return dataset, which will be used as regression input
    combined_ret = pd.merge(ff_ret, train, how='right', on=['YearMonth'])
    # calculates the excess return of tickers by subtracting the returns by risk free rate
    combined_ret['ticker_excess_ret'] = combined_ret['ticker_ret'] - combined_ret['RF']
    #append dates
    month_list=["01","02","03","04","05","06","07","08","09","10","11","12"]
    month=month_list[month-1]
#     year = year + 3
    str_date = str(year)+str(month)
    dates.append(str_date)
        
    sd_dic, output = all_regression(combined_ret)

    total_train_output = total_train_output.append(output)
    test_port = test_port.append(test)
    print(i)

890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085


In [38]:
total_train_output

,YearMonth,mkt_ret,SMB,HML,RF,PERMNO,ticker_ret,weight,ticker_excess_ret,residual,residual_sd,residual_mean,mkt_factor,SMB_factor,HML_factor,alpha,mkt_beta,SMB_beta,HML_beta,r_squared,t_alpha,t_mkt_beta,t_SMB_beta,t_HML_beta,port_mkt_beta,port_SMB_beta,port_HML_beta,residual_weight
22369,200302,-0.0188,-0.0034,-0.0145,0.0009,55976,0.005439,-0.001632,0.004539,0.008655,0.127495,-2.081668e-17,-0.009476,0.001391,-0.001543,0.005511,0.504025,-0.409173,0.106424,0.111226,0.675104,5.504021e-02,0.142104,0.745763,-0.000823,6.678281e-04,-0.000174,-0.002929
22378,200302,-0.0188,-0.0034,-0.0145,0.0009,58094,-0.005859,-0.000271,-0.006759,-0.001035,-0.016474,2.390063e-17,-0.010302,-0.000097,-0.010727,0.015402,0.547966,0.028630,0.739778,0.149046,0.210500,2.567954e-02,0.910080,0.019603,-0.000148,-7.745301e-06,-0.000200,-0.001926
22535,200302,-0.0188,-0.0034,-0.0145,0.0009,77546,0.007267,-0.000133,0.006367,-0.006725,-0.073388,-2.891206e-17,-0.016597,0.001285,0.000147,0.028258,0.882846,-0.377860,-0.010115,0.158761,0.117904,1.453716e-02,0.310878,0.981771,-0.000118,5.036365e-05,0.000001,-0.002163
22372,200302,-0.0188,-0.0034,-0.0145,0.0009,56275,0.024905,0.001168,0.024005,0.034612,0.635399,1.850372e-17,-0.011043,0.001092,-0.009033,0.008378,0.587381,-0.321066,0.622970,0.303162,0.428799,6.826077e-03,0.150819,0.023124,0.000686,-3.749367e-04,0.000727,0.000670
22547,200302,-0.0188,-0.0034,-0.0145,0.0009,78916,0.021466,0.005809,0.020566,0.027355,0.233694,7.170190e-17,-0.007195,-0.000618,-0.001096,0.002120,0.382726,0.181864,0.075555,-0.055193,0.925389,3.870405e-01,0.700597,0.893723,0.002223,1.056379e-03,0.000439,0.005491
22297,200302,-0.0188,-0.0034,-0.0145,0.0009,42796,-0.070514,0.003135,-0.071414,-0.079935,-1.277585,6.167906e-18,-0.002768,-0.000775,0.000472,0.011591,0.147209,0.228016,-0.032562,-0.021332,0.341582,5.324820e-01,0.369589,0.914217,0.000461,7.147958e-04,-0.000102,0.001011
22562,200302,-0.0188,-0.0034,-0.0145,0.0009,79915,0.031146,-0.003362,0.030246,0.052052,0.452981,1.233581e-17,-0.014603,0.000008,-0.011252,0.004042,0.776781,-0.002356,0.776034,0.030385,0.855724,7.920474e-02,0.995946,0.168502,-0.002612,7.922352e-06,-0.002609,-0.004371
22335,200302,-0.0188,-0.0034,-0.0145,0.0009,49680,0.058785,-0.002327,0.057885,0.067011,0.811174,3.083953e-18,-0.020246,0.000547,-0.008945,0.019518,1.076904,-0.160913,0.616884,0.229724,0.227231,1.406069e-03,0.629981,0.129096,-0.002506,3.743856e-04,-0.001435,-0.003625
22343,200302,-0.0188,-0.0034,-0.0145,0.0009,51377,0.297727,-0.005624,0.296827,0.335198,1.994915,4.934325e-17,-0.043636,-0.005423,-0.003468,0.014157,2.321079,1.595111,0.239175,0.414393,0.663583,7.957507e-04,0.023947,0.768413,-0.013054,-8.971370e-03,-0.001345,-0.006940
22454,200302,-0.0188,-0.0034,-0.0145,0.0009,69032,-0.027705,-0.001106,-0.028605,-0.011650,-0.199906,-4.240435e-17,-0.040406,0.000310,0.000317,0.022824,2.149247,-0.091208,-0.021853,0.789502,0.049586,2.955927e-11,0.698508,0.938130,-0.002378,1.009211e-04,0.000024,-0.000893


In [37]:
export_csv1 = total_train_output.to_csv ('train_output4.csv', index = None, header=True)
export_csv2 = test_port.to_csv ('test_output4.csv', index = None, header=True)